# 교차 검증과 그리드 서치

## 과대적합 vs 과소적합
* 훈련세트의 점수는 좋은데 테스트세트의 점수가 나쁘면 모델이 훈련세트에 과대적합(overfitting) 되었다고 하고
    - 훈련세트에만 잘 맞는 모델이라 테스트 세트에 대한 예측을 만들었을 때 잘 동작하지 않는 현상
* 훈련세트보다 테스트세트의 점수가 높거나, 두 점수가 너무 낮은 경우는 모델이 훈련세트에 과소적합(underfitting) 되었다고 한다.
    - 모델이 너무 단순하여 훈련세트에 적절히 훈련되지 않는 현상

## 검증 세트
* 테스트 세트를 사용하지 않으면 모델이 과대적합인지 과소적합인지 판단하기 어렵다
* 테스트 세트를 사용하지 않고 측정하는 방법은 훈련세트를 다시 나누면 된다. 이 데이터를 검증세트(validation set) 라고 한다
* 훈련세트의 모델로 학습을 하고, 검증세트로 모델을 평가한다 
    - 테스트 하고 싶은 매개변수를 바꾸면서 가장 좋은 모델을 찾는다
    - 찾은 매개변수를 사용해 훈련세트와 검증세트를 합쳐 전체 훈련데이터에서 모델을 다시 학습을 합니다.
    - 마지막에 테스트 세트에서 최종 점수를 평가한다

In [1]:
import pandas as pd

wine = pd.read_csv('wine.csv')
print(wine.shape)
wine.head()

(6497, 4)


,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [2]:
#독립변수
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
print(type(data), data.shape)
#종속변수
target = wine['class'].to_numpy()
print(type(target), target.shape)

<class 'numpy.ndarray'> (6497, 3)
<class 'numpy.ndarray'> (6497,)


In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data, target, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(5197, 3) (5197,)
(1300, 3) (1300,)


* 훈련세트 데이터를 훈련세트와 검증세트로 분할한다
* 훈련세트 
    - 독립변수 : sub_input
    - 종속변수 : sub_target
* 검증세트
    - 독립변수 : val_input
    - 종속변수 : val_target

In [8]:
sub_input, val_input, sub_target, val_target = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [9]:
print(sub_input.shape, sub_target.shape)
print(val_input.shape, val_target.shape)

(4157, 3) (4157,)
(1040, 3) (1040,)


In [11]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=42)
#학습
model.fit(sub_input, sub_target)

#훈련세트에 과대적합(overfitting) 되어 있음
print(model.score(sub_input, sub_target))
print(model.score(val_input, val_target))

0.9971133028626413
0.864423076923077


## 교차 검증
* 교차검증(cross validation)은 검증세트를 떼어 내고 평가하는 과정을 여러번 반복한다
* 훈련세트를 세 부분으로 나누어서 교차 검증을 수행하는 것을 3-폴드 교차 검증이라고 한다
    - k-폴드 교차검증(k-Fold cross validation)이라고 한다
* fit_time과 score_time은 모델을 훈련하는 시간과 검증시간을 의미한다
* cross_validate()는 5-폴드 교차 검증을 수행한다

In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate

model = DecisionTreeClassifier(random_state=42)
scores = cross_validate(model, X_train, y_train)
print(scores)

{'fit_time': array([0.00399756, 0.00762701, 0.0065167 , 0.00358701, 0.00585103]), 'score_time': array([0.        , 0.00066924, 0.00050092, 0.00102019, 0.00078893]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


* 교차검증의 최종 점수는 test_score에 담긴 5개 점수의 평균하면 알 수 있다
* test_score이지만 검증 폴드의 점수이다

In [13]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


* cross_validate()는 회귀 모델일 경우 KFold를 사용하고
* 분류 모델일 경우 클래스를 골고루 나누기 위해 StratifiedKFold를 사용한다

In [15]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(model, X_train, y_train, cv=StratifiedKFold())
print(scores)
print(np.mean(scores['test_score']))

{'fit_time': array([0.00533819, 0.0066731 , 0.00486684, 0.00552678, 0.00635433]), 'score_time': array([0.        , 0.00099897, 0.00114465, 0.00099802, 0.        ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
0.855300214703487


* 훈련세트를 섞은 후에 10-폴드 교차 검증을 수행한다

In [17]:
stFold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(model, X_train, y_train, cv=stFold)
print(np.mean(scores['test_score']))

0.8574181117533719


## GridSearchCV를 사용한 하이퍼파라미터 튜닝
* 모델이 학습하는 파라미터를 모델 파라미터라고 부른다
* 모델이 학습할 수 없어서 사용자가 지정해야 하는 파라미터를 하이퍼파라미터 라고 부른다
* GridSearchCV는 하이퍼파라미터 탐색과 교차검증을 한 번에 수행한다. 
    - cross_validate() 함수를 호출할 필요가 없다

In [21]:
# max_depth(트리의 높이) 와 같이 가지치기를 할때 사용하는 매개변수 min_impurity_decrease(최소불순도)
from sklearn.model_selection import GridSearchCV
#min_impurity_decrease
params = {'min_impurity_decrease':[0.0001,0.0002,0.0003,0.0004,0.0005]}

model = DecisionTreeClassifier(random_state=42)
gridSCV = GridSearchCV(model, params, n_jobs=-1)

gridSCV

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [22]:
gridSCV.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [24]:
best = gridSCV.best_estimator_
best.score(X_train, y_train)

0.9615162593804117

In [25]:
#최적의 파라미터
gridSCV.best_params_

{'min_impurity_decrease': 0.0001}

* 각 매개변수에서 수행한 교차검증의 평균점수는 cv_results_ 속성의 mean_test_score에 저장되어 있다
* 5번의 교차검증으로 얻은 점수는 아래와 같다

In [26]:
gridSCV.cv_results_

{'mean_fit_time': array([0.00936913, 0.00717645, 0.00660725, 0.00643005, 0.00597849]),
 'std_fit_time': array([0.00428614, 0.00263712, 0.00146288, 0.00210573, 0.00142304]),
 'mean_score_time': array([0.00118313, 0.00090623, 0.00152245, 0.00074081, 0.00077739]),
 'std_score_time': array([0.00014863, 0.00049694, 0.00078812, 0.00023447, 0.00023134]),
 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'min_impurity_decrease': 0.0001},
  {'min_impurity_decrease': 0.0002},
  {'min_impurity_decrease': 0.0003},
  {'min_impurity_decrease': 0.0004},
  {'min_impurity_decrease': 0.0005}],
 'split0_test_score': array([0.86923077, 0.87115385, 0.86923077, 0.86923077, 0.86538462]),
 'split1_test_score': array([0.86826923, 0.86346154, 0.85961538, 0.86346154, 0.86923077]),
 'split2_test_score': array([0.8825794 , 0.87680462, 0.87584216, 0.88161

In [27]:
gridSCV.cv_results_['mean_test_score']

array([0.86819297, 0.86453617, 0.86492226, 0.86780891, 0.86761605])

* numpy의 argmax()를 사용하면 가장 큰 값의 인덱스를 추출할 수 있다
* 그 인덱스를 사용해 params 키에 저장된 매개변수를 출력할 수 있다

In [29]:
best_index = np.argmax(gridSCV.cv_results_['mean_test_score'])
print(best_index)
print(gridSCV.cv_results_['params'][best_index])

0
{'min_impurity_decrease': 0.0001}


* min_impurity_decrease는 노드를 분할하기 위한 불순도 감소 최소량를 지정한다
* max_depth로 트리의 깊이를 제한하고
* min_samples_split으로 노드를 분할하기 위한 최소 샘플수

In [30]:
params = {
    'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
    'max_depth': range(5,20,1),
    'min_samples_split': range(2,100,10)
}
gridCV = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gridCV.fit(X_train, y_train)
          

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [31]:
gridCV.best_params_

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}

In [33]:
#gridCV.cv_results_
np.max(gridCV.cv_results_['mean_test_score'])

0.8683865773302731

In [35]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(5197, 3) (5197,)
(1300, 3) (1300,)


In [36]:
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

#overfitting
print('훈련', model.score(X_train, y_train))
print('테스트', model.score(X_test, y_test))

훈련 0.996921300750433
테스트 0.8584615384615385


In [37]:
model = DecisionTreeClassifier(random_state=42, max_depth=14, min_impurity_decrease=0.0004, min_samples_split=12)
model.fit(X_train, y_train)

print('훈련', model.score(X_train, y_train))
print('테스트', model.score(X_test, y_test))

훈련 0.892053107562055
테스트 0.8615384615384616


### 랜덤 서치
* 랜덤서치는 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링 할 수 있는 확률분포를 전달한다
* scipy의 uniform과 randint 클래스는 모두 주어진 범위에서 고르게 값을 추출해준다
 - 즉, 균등분포에서 샘플링한다 
 - randint는 정수값을, uniform은 실수값을 추출한다
* 임의로 샘플링 하므로 아래와 결과가 다를 수도 있다  

array([4, 7, 6, 7, 4, 8, 0, 0, 8, 7])

* 1000개를 샘플링해서 각 숫자의 개수를 세어본다

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([109, 108,  93,  93, 100, 108,  95,  92,  96, 106], dtype=int64))

array([0.17618235, 0.59150809, 0.97593258, 0.78709444, 0.25405458,
       0.03083526, 0.43932543, 0.55865703, 0.6381422 , 0.64808261])

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001F09413E100>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001F0942BCF40>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001F0942BC9A0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001F0943AC520>},
                   random_state=42)

{'max_depth': 45, 'min_impurity_decrease': 0.000404781258158029, 'min_samples_leaf': 6, 'min_samples_split': 9}


0.8693486710594508


0.8576923076923076


## 확인문제
* splitter=random 매개변수를 추가하고 다시 훈련
* splitter 매개변수는 기본값은 'best'로 최선의 분할을 찾는다
 - random이면 무작위로 분할 한 다음 가장 좋은 것을 고른다

In [44]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(X_train, y_train)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001BBCABBC6A0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001BBCAB379D0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001BBCABBCB80>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001BBCABBC7C0>},
                   random_state=42)

In [45]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(X_test, y_test))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077
